In [3]:
!pip install SpeechRecognition pydub fpdf
!pip install ffmpeg

import os
import speech_recognition as sr
from pydub import AudioSegment
from fpdf import FPDF

ERROR: Operation cancelled by user


ImportError: cannot import name 'FPDF' from 'ffmpeg' (/usr/local/lib/python3.12/dist-packages/ffmpeg/__init__.py)

In [ ]:
audio_path = "/content/audio.mp4"
segment_dir = "/content/segmentos"
os.makedirs(segment_dir, exist_ok=True)

audio = AudioSegment.from_file(audio_path)

In [ ]:
segment_length_ms = 60 * 1000
segments = [
    audio[i:i + segment_length_ms]
    for i in range(0, len(audio), segment_length_ms)
]

recognizer = sr.Recognizer()
transcripcion = ""

for i, segment in enumerate(segments):
    segment_path = f"{segment_dir}/segmento_{i}.wav"
    segment.export(segment_path, format="wav")

    with sr.AudioFile(segment_path) as source:
        audio_data = recognizer.record(source)
        try:
            texto = recognizer.recognize_google(audio_data, language="es-ES")
            transcripcion += texto + "\n\n"
        except sr.UnknownValueError:
            transcripcion += "[No se pudo transcribir este segmento]\n\n"


In [ ]:
with open("/content/transcripcion.txt", "w", encoding="utf-8") as f:
    f.write(transcripcion)

In [ ]:
pdf = FPDF()
pdf.add_page()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.set_font("Arial", size=12)
pdf.multi_cell(0, 8, transcripcion)

pdf.output("/content/transcripcion.pdf")